In [1]:
import math

import scipy

import numpy as np

R = PolynomialRing(QQ,30,"X")

z = R.gens()

In [2]:

##Definitions polynomials.

#We represent here a polynomial P by a list L (called "polynomial list") of lists of the form [c,E], where c is a real number and 
# E is a list of positive integers. Each list of the form [c,E] in L corresponds to the monomial in P with coefficient c
# and whose variables are the one with indices the elements of E, with multiplicities.
# For instance, the list [2,[2,2,3]] corresponds to the monomial 2X_2X_2X_3.

def product(k, L): 

# take a polynomial list L representing a polynomial P and an integer k, and gives back the polynomial list 
# representing the multiplication of P with the monomial X_k.
    S = list(L)
    
    for E in S: 
        
        E[1] = sorted(E[1] + [k])
    
    return(S)

def  scalaire(c,L): 
    
# take a polynomial list L representing a polynomial P and an real number c, and gives back the polynomial list 
# representing the multiplication of P with the number c.
    
    S = list(L)
    
    for E in S:
        
        E[0] = c*E[0]
        
    return(S)

def D(L): 

# take a polynomial list representing a polynomial P and gives back the polynomial list representing the
# polynomial D(P) (defined in Definition 3.2).
    
    S = []
    
    for E in L:
        
        if  E[0] != 0:
        
            S = S + Dintermed(E)
        
    return(S) 

def Dintermed(E):
    
# auxiliary function for D.
    
    L = []
    
    l = len(E[1])
    
    for i in range(l):
        
        W = list(E[1])
        
        W[i] = W[i] + 1
        
        L = L + [[E[0],sorted(W)]]
        
    return L 

def D2(L): 

# take a polynomial list representing a polynomial P and gives back the polynomial list representing the
# polynomial D_2(P) (defined in Definition 3.2).
    
    S = []
    
    for E in L:
        
        if E[0] != 0 :
        
            S = S + Dintermed2(E)
        
    return(S)

def Dintermed2(E):

# auxiliary function for D2.
    
    rep = []
    
    l = len(E[1])
    
    for i in range(l):
        
        W = list(E[1])
        
        W[i] = W[i] + 2
        
        rep = rep + [[-E[0]/2,sorted(W)]]
        
    return rep 



def H(L):

# take a polynomial list representing a polynomial P and gives back the polynomial list representing the
# polynomial H(P) (defined in Definition 3.3).
    
    S = []
    
    for E in L:
        
        if E[0] != 0:
        
            for j in range(len(E[1])):
            
                auxbis = []
                    
                for i in range(E[1][j]+1):
                    
                    auxbis = auxbis + [[-(1/2)*binomial(E[1][j],i)*E[0], [i+1,1+E[1][j]-i]]]
                    
                    
                for F in auxbis:
                
                        F[1] = sorted(F[1] + Hintermed1(j,E[1]))
                
                S = S + auxbis
    
    return S

def Hintermed1(k,lst): 
    
# where k <= len(lst) ; auxiliary function for H.
    
    rep = []
    
    n = len(lst)
    
    for i in range(n):
        
        if i != k :
            
            rep = rep + [lst[i]]
    
    return rep


##Algorithms to calculate the partition of an integer. 

def successeur(a):
    
    k = len(a)
    
    if k==1:
        
        return a
    
    y = a[k-1]-1
    
    x = a[k-2]+1
    
    q = y//x
    
    b = a[:k-2]+q*[x]+[y-(q-1)*x]
    
    return b

def partitions(n):
    
    a = n*[1]
    
    L = [a]
    
    while a!=[n]:
        
        a = successeur(a)
        
        L += [a]
    
    return L

def partitionR(n): 
    
#calculates the partitions appearing for the sum in the polynomial R_n.
    
    L = partitions(2*n)
    
    S = []
    
    for k in L:
        
        if not ap1(k):
            
            if k[-1] <= n-1:
                
                if len(k) <= n:
            
                    S.append(k)
    
    
    
    return S

def ap1(k):
    
    rep = False
    
    for i in k:
        
        if i==1 :
            
            return True
    
    return rep

##Algorithms to compute R_n.

def initial(n): 
    
#list of the monomials appearing in R_n.
    
    L = partitionR(n)
    
    S = []
    
    for k in L:
        
        S.append([0,k])
        
    return S

def Rbis(n): 
    
#compute R_n
    
    rep = [[-2,[2,2,2]]]
    
    if n == 3: return rep
    
    for k in range(3,n): # n is greater than or equal to 4.
        
        aux = []
        
        S = initial(k+1)
        
        m = len(S)
        
        #compute A_n
        
        for i in range(1,k):
            
            aux.append([-binomial(k,i),sorted([i+1,k,k+1-i])])
        
        ajout(aux,S,m,k)
        
        #compute [\Delta,V_n]
        
        aux = scalaire(1/2,D(D(rep))) + D2(rep)
        
        ajout(aux,S,m,k)
        
        #compute [\Gamma,V_n]
        
        aux = []
        
        aux = product(1,D(rep)) + H(rep)
        
        ajout(aux,S,m,k)
        
        #setting induction
        
        rep = S
        
    return rep

def ajout(L,S,m,k): 
    
# auxiliary function.
    
    for C in L:
        
        i = 0
        
        boue = True
        
        if C[1][-1] == k+1 or C[1][0] == 1:
            
            boue = False
        
        while boue and i<m and C[1] != S[i][1]:
            
            i = i + 1
            
        if boue == True:
            
            S[i][0] = S[i][0] + C[0]
 
            
def convert_aux(E): 
    
# auxiliary function for convert.
    
    a = E[0]
    
    b = E[1]
    
    P = 1
    
    for i in b:
        
        P = P*z[i]
        
    return a*P

def convert(L): 
    
# converts a polynomial list L into the corresponding polynomial.
    
    P = 0
    
    for E in L:
        
        P = P + convert_aux(E)
    
    return P 


    
def R_tilde(n): 
    
# We only add X_l^2 to R_l ; computes Rtilda_n.
    
    rep = Rbis(n) + [[1,[n,n]]]
    
    #rep = convert(rep)
    
    return rep

def R_nonzero(n): 
    
# remove monomial with odd indices in R_n.
    
    L = R_tilde(n)
    
    S = []
    
    for k in L:
        
        if k[0] != 0 and no_odd(k[1]):
            
            S.append(k)
        
    return convert(S)
        
def no_odd(L): 
    
# auxiliary function for R_nonzero.
        
    rep = True
        
    for k in L:
            
        if k%2 != 0:
                
            return False
            
    return rep

def aN(n): 
    
# compute a_n.
    
    aux = R_tilde(n+1)
    
    S = []
    
    for E in aux:
        
        if E[0] != 0 and E[1][-1] == n:
            
            S.append(E)
        
        
    S = S[1:]
    
    for E in S:
        
        E[1] = E[1][:-1]
    
    return S
    